# Transport problem
From Section 8.1 - tableau



In [1]:
# Transport problem
import numpy as np
from simplex import simplex_solver

# setup TP
source_constraint_list = [75, 125, 100]
destination_constraint_list = [80, 65, 70, 85]
cost_tab = [[464, 513, 654, 867],
            [352, 416, 690, 791],
            [995, 682, 388, 685]
            ]

In [2]:

n_source = len(source_constraint_list)
n_destination = len(destination_constraint_list)
n_decision_var = n_source * n_destination


def convert_2d_1d(i_source, i_destination):
    return i_source * n_destination + i_destination


# setup LP
c = []
for i_src in range(n_source):
    c += cost_tab[i_src]
c = - np.array(c)

b = source_constraint_list + destination_constraint_list

A = []
for i_src in range(n_source):
    row = [0] * n_decision_var
    for i_dst in range(n_destination):
        row[convert_2d_1d(i_src, i_dst)] = 1
    A += [row]
for i_dst in range(n_destination):
    row = [0] * n_decision_var
    for i_src in range(n_source):
        row[convert_2d_1d(i_src, i_dst)] = 1
    A += [row]

# initial tableau with big M
M = 10000.

# first row of the tableau
tab01 = - np.array(c).reshape((1, len(c))) - M * np.ones((1, len(A))) @ np.array(A)
tab0 = np.append(np.array([1]).reshape((1, 1)), tab01, axis=1)
tab0 = np.append(tab0, np.zeros((1, len(b))), axis=1)
tab0 = np.append(tab0, - M * np.ones((1, len(b))) @ np.array(b).reshape((len(b), 1)), axis=1)

tab1 = np.append(np.zeros((len(b), 1)), np.array(A), axis=1)
tab1 = np.append(tab1, np.eye(len(b)), axis=1)
tab1 = np.append(tab1, np.array(b).reshape((len(b), 1)), axis=1)

tab = np.append(tab0, tab1, axis=0)

In [3]:
print(tab)

[[1.00 -19536.00 -19487.00 -19346.00 -19133.00 -19648.00 -19584.00
  -19310.00 -19209.00 -19005.00 -19318.00 -19612.00 -19315.00 0.00 0.00
  0.00 0.00 0.00 0.00 0.00 -6000000.00]
 [0.00 1.00 1.00 1.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00
  0.00 0.00 0.00 0.00 0.00 0.00 75.00]
 [0.00 0.00 0.00 0.00 0.00 1.00 1.00 1.00 1.00 0.00 0.00 0.00 0.00 0.00
  1.00 0.00 0.00 0.00 0.00 0.00 125.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 1.00 1.00 1.00 0.00
  0.00 1.00 0.00 0.00 0.00 0.00 100.00]
 [0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00
  0.00 0.00 1.00 0.00 0.00 0.00 80.00]
 [0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00
  0.00 0.00 0.00 1.00 0.00 0.00 65.00]
 [0.00 0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00
  0.00 0.00 0.00 0.00 1.00 0.00 70.00]
 [0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00
  0.00 0.00 0.00 0.00 0.00 1.00 85.00]]


In [4]:
simplex_solver(tab, display=1)

7 constraints and 19 variables
pivots are: [13, 14, 15, 16, 17, 18, 19]
initial tableau is:
[[1.00 -19536.00 -19487.00 -19346.00 -19133.00 -19648.00 -19584.00
  -19310.00 -19209.00 -19005.00 -19318.00 -19612.00 -19315.00 0.00 0.00
  0.00 0.00 0.00 0.00 0.00 -6000000.00]
 [0.00 1.00 1.00 1.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00
  0.00 0.00 0.00 0.00 0.00 0.00 75.00]
 [0.00 0.00 0.00 0.00 0.00 1.00 1.00 1.00 1.00 0.00 0.00 0.00 0.00 0.00
  1.00 0.00 0.00 0.00 0.00 0.00 125.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 1.00 1.00 1.00 0.00
  0.00 1.00 0.00 0.00 0.00 0.00 100.00]
 [0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00
  0.00 0.00 1.00 0.00 0.00 0.00 80.00]
 [0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00
  0.00 0.00 0.00 1.00 0.00 0.00 65.00]
 [0.00 0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00
  0.00 0.00 0.00 0.00 1.00 0.00 70.00]
 [0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00